In [33]:
import pandas as pd

In [34]:
path_trans = (r"C:\Users\ASUS\Documents\Project\Momo case study\Data\processed\df_transactions.csv")
path_users = (r"C:\Users\ASUS\Documents\Project\Momo case study\Data\processed\df_users.csv")
path_products = (r"C:\Users\ASUS\Documents\Project\Momo case study\Data\processed\df_products.csv")

In [35]:
df_products = pd.read_csv(path_products, encoding = 'utf-8')
df_users = pd.read_csv(path_users, encoding = 'utf-8')
df_trans = pd.read_csv(path_trans, encoding = 'utf-8' )


In [36]:
print(df_trans.duplicated('order_id').sum())

0


In [37]:
print(df_users.duplicated('user_id').sum())

0


In [38]:
df_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13495 entries, 0 to 13494
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          13495 non-null  int64  
 1   order_id         13495 non-null  int64  
 2   Date             13495 non-null  object 
 3   Amount           13495 non-null  float64
 4   Merchant_id      13495 non-null  int64  
 5   Purchase_status  13495 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 632.7+ KB


In [39]:
df_trans = df_trans.merge(
    df_products, on="Merchant_id", how="left"
)

df_trans["Revenue"] = df_trans["Amount"] * df_trans["Rate_pct"]

In [40]:
jan_2020_revenue = df_trans.loc[
    (df_trans['Date'] >= "2020-01-01") &
    (df_trans['Date'] < "2021-01-01"), 
    "Revenue"].sum()


In [41]:
print(jan_2020_revenue)

1871272702.0


In [42]:
df_trans['Date'] = pd.to_datetime(df_trans['Date'])


In [43]:
df_users['First_tran_date'] = pd.to_datetime(df_users['First_tran_date'])


In [44]:
print(df_users['First_tran_date'].isnull().sum())

0


In [45]:
most_profitable_month = (
    df_trans
    .groupby(df_trans["Date"].dt.to_period("M"))["Revenue"]
    .sum()
    .idxmax()
)
total_for_month = (
    df_trans
    .groupby(df_trans['Date'].dt.to_period("M"))["Revenue"]
    .sum()
    .max()
)
print(most_profitable_month,total_for_month)

2020-09 170220000.0


In [46]:
weekday_avg = (
    df_trans
    .groupby(df_trans["Date"].dt.day_name())["Revenue"]
    .mean()
)
most_profitable_day = weekday_avg.idxmax()
least_profitable_day = weekday_avg.idxmin()

print(most_profitable_day, least_profitable_day)

Wednesday Monday


In [47]:
df_users.rename(columns={'User_id': 'user_id'}, inplace=True)

In [48]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13390 entries, 0 to 13389
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   user_id          13390 non-null  int64         
 1   First_tran_date  13390 non-null  datetime64[ns]
 2   Location         13390 non-null  object        
 3   Age              13390 non-null  object        
 4   Gender           13390 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 523.2+ KB


In [49]:
df_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13495 entries, 0 to 13494
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   user_id          13495 non-null  int64         
 1   order_id         13495 non-null  int64         
 2   Date             13495 non-null  datetime64[ns]
 3   Amount           13495 non-null  float64       
 4   Merchant_id      13495 non-null  int64         
 5   Purchase_status  13495 non-null  int64         
 6   Merchant_name    13495 non-null  object        
 7   Rate_pct         13495 non-null  int64         
 8   Revenue          13495 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(5), object(1)
memory usage: 949.0+ KB


In [50]:
df_trans_tx = df_trans.merge(
    df_users,
    on="user_id",
    how="left"
)

In [51]:
df_trans_tx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13495 entries, 0 to 13494
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   user_id          13495 non-null  int64         
 1   order_id         13495 non-null  int64         
 2   Date             13495 non-null  datetime64[ns]
 3   Amount           13495 non-null  float64       
 4   Merchant_id      13495 non-null  int64         
 5   Purchase_status  13495 non-null  int64         
 6   Merchant_name    13495 non-null  object        
 7   Rate_pct         13495 non-null  int64         
 8   Revenue          13495 non-null  float64       
 9   First_tran_date  13495 non-null  datetime64[ns]
 10  Location         13495 non-null  object        
 11  Age              13495 non-null  object        
 12  Gender           13495 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(5), object(4)
memory usage: 1.3+ MB


In [52]:
df_trans_tx["Age"] = df_trans_tx["Age"].astype("category")
df_trans_tx["Age"].value_counts()

Age
23_to_27    3430
28_to_32    2816
unknown     2529
33_to_37    1754
18_to_22    1704
>37         1262
Name: count, dtype: int64

In [53]:
df_trans_tx["Type_user"] = (
    df_trans_tx["Date"].dt.to_period("M") ==
    df_trans_tx["First_tran_date"].dt.to_period("M")
)

df_trans_tx["Type_user"] = df_trans_tx["Type_user"].map({
    True: "New",
    False: "Current"
})

In [54]:
new_users_dec_2020 = (
    df_trans_tx[
        (df_trans_tx["Type_user"] == "New") &
        (df_trans_tx["Date"].dt.to_period("M") == "2020-12")
    ]["user_id"]
    .nunique()
)

new_users_dec_2020

76

In [55]:
df_trans_tx[df_trans_tx["Type_user"] == "New"] \
    .sort_values(["user_id","Date"]) \
    .head(20)

,user_id,order_id,Date,Amount,Merchant_id,Purchase_status,Merchant_name,Rate_pct,Revenue,First_tran_date,Location,Age,Gender,Type_user
9748,644824,7362695016,2020-09-29,20000.0,12,0,Viettel,2,40000.0,2020-09-29,Other Cities,unknown,MALE,New
13286,967948,8672800182,2020-12-28,200000.0,12,1,Viettel,2,400000.0,2020-12-10,HCMC,33_to_37,FEMALE,New
1694,2174908,4856496186,2020-02-22,100000.0,12,0,Viettel,2,200000.0,2020-02-22,Other Cities,unknown,FEMALE,New
3640,2338603,5378659071,2020-04-18,100000.0,14,0,Vinaphone,4,400000.0,2020-04-06,Other Cities,28_to_32,FEMALE,New
11085,2606556,7873025220,2020-11-02,10000.0,12,0,Viettel,2,20000.0,2020-11-02,Other Cities,18_to_22,FEMALE,New
3891,2930619,5448895812,2020-04-26,20000.0,14,0,Vinaphone,4,80000.0,2020-04-26,Other Cities,unknown,FEMALE,New
566,3002366,4345423020,2020-01-19,20000.0,13,0,Mobifone,3,60000.0,2020-01-16,Other Cities,unknown,MALE,New
11905,3139148,8178022062,2020-11-24,50000.0,13,0,Mobifone,3,150000.0,2020-11-02,HCMC,33_to_37,MALE,New
8151,3359053,6761897644,2020-08-20,100000.0,12,0,Viettel,2,200000.0,2020-08-10,Other Cities,unknown,FEMALE,New
8329,3373534,6821951120,2020-08-25,100000.0,12,0,Viettel,2,200000.0,2020-08-17,Other Cities,unknown,MALE,New


In [56]:
df_trans_tx.dtypes

user_id                     int64
order_id                    int64
Date               datetime64[ns]
Amount                    float64
Merchant_id                 int64
Purchase_status             int64
Merchant_name              object
Rate_pct                    int64
Revenue                   float64
First_tran_date    datetime64[ns]
Location                   object
Age                      category
Gender                     object
Type_user                  object
dtype: object

In [58]:
df_trans_tx.to_csv('final.csv',index=False,encoding= 'utf-8-sig')